[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andikatedja/web-scraping/blob/master/Scraping%20Member%20JKT48.ipynb)
# Scraping sederhana member JKT48 dari website JKT48
## Menampilkan list nama tiap tim, dan jumlah member tiap tim
Notebook ini saya buat untuk saya belajar jupyter notebooks

In [1]:
import bs4
import requests

## Scraping member yang telah masuk ke dalam tim J K T
Mengambil informasi dari website member JKT48  

In [2]:
res = requests.get('https://www.jkt48.com/member/list?lang=id')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('div.profilename > a')
# Print isi dari elems
print(elems)

[<a href="/member/detail/id/124?lang=id">Adriani<br/>Elisabeth</a>, <a href="/member/detail/id/220?lang=id">Amanina<br/>Afiqah</a>, <a href="/member/detail/id/218?lang=id">Aninditha<br/>Rahma<br/>Cahyadi</a>, <a href="/member/detail/id/186?lang=id">Ariella<br/>Calista<br/>Ichwan</a>, <a href="/member/detail/id/200?lang=id">Azizi<br/>Asadel</a>, <a href="/member/detail/id/134?lang=id">Cindy Hapsari<br/>Maharani<br/>Pujiantoro Putri</a>, <a href="/member/detail/id/169?lang=id">Diani<br/>Amalia<br/>Ramadhani</a>, <a href="/member/detail/id/171?lang=id">Eve<br/>Antoinette<br/>Ichwan</a>, <a href="/member/detail/id/76?lang=id">Feni<br/>Fitriyanti</a>, <a href="/member/detail/id/77?lang=id">Fransisca<br/>Saraswati<br/>Puspa Dewi</a>, <a href="/member/detail/id/11?lang=id">Frieska<br/>Anastasia<br/>Laksani</a>, <a href="/member/detail/id/202?lang=id">Gabriel<br/>Angelina</a>, <a href="/member/detail/id/12?lang=id">Gabriela<br/>Margareth<br/>Warouw</a>, <a href="/member/detail/id/148?lang=id">

## Hilangkan tag html
Kita akan mengambil isi dari html dengan menggunakan get_text()  
Maka tampil nama nama member yang diambil dari website jkt48, tidak ada tag html seperti pada hasil eksekusi diatas

In [3]:
for member in elems:
    print(member.get_text(separator=' ').strip())

Adriani Elisabeth
Amanina Afiqah
Aninditha Rahma Cahyadi
Ariella Calista Ichwan
Azizi Asadel
Cindy Hapsari Maharani Pujiantoro Putri
Diani Amalia Ramadhani
Eve Antoinette Ichwan
Feni Fitriyanti
Fransisca Saraswati Puspa Dewi
Frieska Anastasia Laksani
Gabriel Angelina
Gabriela Margareth Warouw
Melati Putri Rahel Sesilia
Nabila Fitriana
Nadila Cindi Wantari
Ni Made Ayu Vania Aurellia
Riska Amelia Putri
Rona Anggreani
Sania Julia Montolalu
Anastasya Narwastu Tety Handuran
Angelina Christy
Aninditha Rahma Cahyadi
Beby Chaesara Anadila
Fidly Immanda Azzahra
Gabryela Marcelina
Gita Sekar Andarini
Helisma Putri
Jinan Safa Safira
Kandiya Rafa Maulidita
Maria Genoveva Natalia Desy Purnamasari Gunawan
Mutiara Azzahra
Nurhayati
Rinanda Syahputri
Shani Indira Natio
Shania Gracia
Yessica Tamara
Zahra Nur
Amirah Fatin
Aninditha Rahma Cahyadi
Aurel Mayori
Cornelia Vanisa
Dhea Angelia
Febriola Sinambela
Fiony Alveria
Flora Shafiq
Freya Jayawardana
Jessica Chandra
Jesslyn Callista
Lulu Salsabila
Nyimas

## Filter member berdasarkan tim
Sekarang agak sedikit tricky. Membagi member berdasarkan tim dan memasukkannya ke dalam list.  
Dapat dilihat bahwa dari hasil eksekusi diatas, nama ditampilkan berurut berdasarkan huruf depan.  
Jadi setiap tim baru maka akan dimulai dengan huruf A, tetapi sebelumnya harus melewati huruf huruf yang bukan huruf A.  
Perhatikan julie dan tasya  
>Sania Julia Montolalu -> Urutan terakhir untuk tim J  
>Anastasya Narwastu Tety Handuran -> Urutan pertama untuk tim KIII  

Berikut ini kode untuk memfilter tim ke dalam tim J

### Filter tim J

In [4]:
# Buat variabel bantu untuk menentukan apakah huruf pertama A atau tidak
notA = False
teamJ = []
for member in elems:
    # Cek apakah saat looping kedua dst, ketemu huruf A.
    # Adriani Elisabeth -> looping pertama huruf A
    # Amanina Afiqah -> Kedua ketemu huruf A juga jadi tetep lanjut looping -> Looping ini masuk ke X
    # ...
    # Cindy Hapsari Maharani Pujiantoro Putri -> Cindy bukan lagi A, jadikan variabel notA menjadi True
    # Sampai Sania Julia Montolalu looping akan masuk ke X
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        # Append ke dalam list teamJ
        teamJ.append(member.get_text(separator=' ').strip())
    else:
        # Cek apakah looping kembali lagi ke A dan notA True
        # Sehingga notA menjadi patokan apakah ketemu A lagi, jika ketemu maka break
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            # X
            # Lanjut looping jika looping tidak huruf A
            teamJ.append(member.get_text(separator=' ').strip())

In [5]:
# Print dan hitung total member tim J
print('Member Team J : \n')
for member in teamJ:
    print(member)
print('\n===Total Member Team J : ' + str(len(teamJ)))

Member Team J : 

Adriani Elisabeth
Amanina Afiqah
Aninditha Rahma Cahyadi
Ariella Calista Ichwan
Azizi Asadel
Cindy Hapsari Maharani Pujiantoro Putri
Diani Amalia Ramadhani
Eve Antoinette Ichwan
Feni Fitriyanti
Fransisca Saraswati Puspa Dewi
Frieska Anastasia Laksani
Gabriel Angelina
Gabriela Margareth Warouw
Melati Putri Rahel Sesilia
Nabila Fitriana
Nadila Cindi Wantari
Ni Made Ayu Vania Aurellia
Riska Amelia Putri
Rona Anggreani
Sania Julia Montolalu

===Total Member Team J : 20


### Filter tim KIII
Tekniknya sama seperti tim J, tapi mulai looping nya dari urutan terakhir tim J  
Jadi variabel elems nya dimulai dari panjang (len) dari variabel teamJ dan diakhiri dari panjang elems itu sendiri

In [6]:
notA = False
teamKIII = []
# Dapat dilihat elems nya dimulai dan diakhiri seperti ini
# Looping akan start dari Anastasya Narwastu Tety Handuran
for member in elems[len(teamJ):len(elems)]:
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        teamKIII.append(member.get_text(separator=' ').strip())
    else:
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            teamKIII.append(member.get_text(separator=' ').strip())

In [7]:
# Print dan hitung member tim KIII
print('Member Team KIII : \n')
for member in teamKIII:
    print(member)
print('\n===Total Member Team KIII : ' + str(len(teamKIII)))

Member Team KIII : 

Anastasya Narwastu Tety Handuran
Angelina Christy
Aninditha Rahma Cahyadi
Beby Chaesara Anadila
Fidly Immanda Azzahra
Gabryela Marcelina
Gita Sekar Andarini
Helisma Putri
Jinan Safa Safira
Kandiya Rafa Maulidita
Maria Genoveva Natalia Desy Purnamasari Gunawan
Mutiara Azzahra
Nurhayati
Rinanda Syahputri
Shani Indira Natio
Shania Gracia
Yessica Tamara
Zahra Nur

===Total Member Team KIII : 18


### Filter tim T
Tekniknya sama seperti tim J dan KIII, tapi mulai looping nya dari urutan terakhir tim KIII  
Jadi variabel elems nya dimulai dari panjang (len) dari variabel teamKIII dan diakhiri dari panjang elems itu sendiri

In [8]:
notA = False
teamT = []
# Looping dimulai dari Amirah Fatin
for member in elems[len(teamJ) + len(teamKIII):len(elems)]:
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        teamT.append(member.get_text(separator=' ').strip())
    else:
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            teamT.append(member.get_text(separator=' ').strip())

In [9]:
print('Member Team T : \n')
for member in teamT:
    print(member)
print('\n===Total Member Team T : ' + str(len(teamT)))

Member Team T : 

Amirah Fatin
Aninditha Rahma Cahyadi
Aurel Mayori
Cornelia Vanisa
Dhea Angelia
Febriola Sinambela
Fiony Alveria
Flora Shafiq
Freya Jayawardana
Jessica Chandra
Jesslyn Callista
Lulu Salsabila
Nyimas Ratu Rafa
Reva Fidela
Tan Zhi Hui Celine
Umega Maulana
Viona Fadrin

===Total Member Team T : 17


## Mencari kennin member
Kennin member adalah member yang masuk ke dalam lebih dari 1 tim  
Pertama tama kita akan mencari kennin 'J K T', 'J K', dan 'J T'  
Kemudian kita akan mencari kennin 'K T'

In [10]:
kennin = 0
kenninMember = {}
# Mencari kennin member 'J K T', 'J K', 'J T'
for i in range(len(teamJ)):
    # Jika nama member di tim J ada juga di tim K dan tim T, maka dia triple team J K T
    if teamJ[i] in teamKIII and teamJ[i] in teamT:
        kennin += 2
        kenninMember[teamJ[i]] = ['J', 'K', 'T']
    # Jika nama member di tim J ada juga di tim K, maka dia double team J K
    elif teamJ[i] in teamKIII:
        kennin += 1
        kenninMember[teamJ[i]] = ['J', 'K']
    # Jika nama member di tim J ada juga di tim T, maka dia double team J T
    elif teamJ[i] in teamT:
        kennin += 1
        kenninMember[teamJ[i]] = ['J', 'T']

# Mencari kennin member 'K T'
for i in range(len(teamKIII)):
    if teamKIII[i] in teamT:
        # Jika nama member di tim K ini tidak masuk ke dalam variabel kenninMember, artinya dia double team K T
        # Jika ada artinya dia sudah masuk ke dalam triple team J K T
        if teamKIII[i] not in kenninMember:
            kennin += 1
            kenninMember[teamKIII[i]] = ['K', 'T']

In [11]:
# Hitung dan print kennin member
print('Kennin : ' + str(len(kenninMember)) + ' Member')
for member, team in kenninMember.items():
    print(member, '=>', team)

Kennin : 1 Member
Aninditha Rahma Cahyadi => ['J', 'K', 'T']


## Scraping member Academy
Mengambil informasi dari website academy JKT48

In [12]:
res = requests.get('https://www.jkt48.com/jkt48-academy/member-academy')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('p.name-member')


In [13]:
# Memasukkan nama nama member ke dalam variabel academyA
academyA = []
for member in elems:
    academyA.append(member.get_text(separator=' ').strip())

In [14]:
# Hitung dan print member academy
print('Member Academy Class A : ')
for member in academyA:
    print(member)
print('\n===Total Member Academy A : ' + str(len(academyA)))

Member Academy Class A : 
Adzana Shaliha
Caithlyn Gwyneth
Chalista Ellysia
Christabel Jocelyn
Cindy Nugroho
Febi Komaril
Febrina Diponegoro
Gabriella Stevany
Indah Cahya
Kathrina Irene
Keisya Ramadhani
Marsha Lenathea
Nabila Gusmarlia
Putri Elzahra
Salma Annisa
Shalza Grasita
Shinta Devi

===Total Member Academy A : 17


## Menghitung total member JKT48
Jumlah tim J + tim K + tim T + academy - kennin  
Variabel kennin berfungsi untuk mengurangi jumlah dari semuanya

In [15]:
# Hitung jumlah total member JKT48
print('\n===Total Member JKT48 : ' +
      str(len(teamJ) + len(teamKIII) + len(teamT) + len(academyA) - kennin))


===Total Member JKT48 : 70
